In [1]:
import cv2
import numpy as np
from IPython.display import display, clear_output
from ipywidgets import widgets
from IPython.display import Image
import threading

# Constants for the gender detection model
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
GENDER_LIST = ['Male', 'Female']

# Load gender detection model (Caffe model)
gender_net = cv2.dnn.readNetFromCaffe(
    'gender_deploy.prototxt', 
    'gender_net.caffemodel'
)

# Load pre-trained face detection model
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Control variables
cap = None
is_running = False

def analyze_frame(frame):
    """Detect faces and classify gender within the frame."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_region = frame[y:y+h, x:x+w]
        blob = cv2.dnn.blobFromImage(face_region, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender_label = GENDER_LIST[gender_preds[0].argmax()]
        
        color = (0, 255, 0) if gender_label == 'Female' else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, gender_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
    return frame

def webcam_stream():
    """Capture video stream, process frames, and display in the notebook."""
    global cap, is_running
    if cap is None:
        cap = cv2.VideoCapture(0)
    
    while is_running:
        success, frame = cap.read()
        if not success:
            break
        frame = cv2.flip(frame, 1)
        processed_frame = analyze_frame(frame)
        
        # Display processed frame in the notebook
        _, buffer = cv2.imencode('.jpg', cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB))
        display(Image(data=buffer.tobytes()))
        clear_output(wait=True)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    display(start_button)

def toggle_webcam(button):
    """Toggle webcam streaming on button click."""
    global is_running
    is_running = not is_running
    if is_running:
        threading.Thread(target=webcam_stream).start()

# Create and display start button
start_button = widgets.Button(description="Start Webcam")
start_button.on_click(toggle_webcam)
display(start_button)


Button(description='Start Webcam', style=ButtonStyle())